In [1]:
from datetime import timedelta
import redcap
import csv
import datetime
import pandas as pd

# REDCap API endpoint and token
data = {
    'token': '744268EC805657595430964A3C0EC970',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://ruff.mrc.gm:8443/redcap/api/',data['token'])

df_data=df.export_records()

In [2]:
df_data=pd.DataFrame(df_data)
ran_data=df_data[['con_participantid_q1','ran_date','randomization_complete']]
ran_data=ran_data[
    (ran_data['randomization_complete']=='2')|
    (ran_data['randomization_complete']=='1')

]

In [5]:

# Convert ran_date column to datetime
ran_data['ran_date'] = pd.to_datetime(ran_data['ran_date'])
#Function to calculate due date based on weeks
def calculate_due_date(row):
 #this get me the number of weekes from ran_date to today and divide it by 7 to get the number of days 
 weeks_elapsed = (datetime.datetime.now() - row['ran_date']).days // 7

 due_date = row['ran_date'] + timedelta(weeks=weeks_elapsed+1) - timedelta(days=1)
 return due_date

# Apply the function to create a new column 'due_date'
ran_data['due_date'] = ran_data.apply(calculate_due_date, axis=1)

print(ran_data[['con_participantid_q1', 'due_date']])

     con_participantid_q1   due_date
1               IN-M-001E 2023-09-13
117             IN-M-002H 2023-09-19
253             IN-M-004F 2023-09-19
392             IN-M-005J 2023-09-17
539             IN-M-007A 2023-09-17
...                   ...        ...
4394            IN-M-095K 2023-09-18
4416            IN-M-097K 2023-09-18
4440            IN-M-098F 2023-09-13
4445            IN-M-099A 2023-09-13
4455            IN-M-105F 2023-09-17

[61 rows x 2 columns]


In [4]:
ran_data.to_csv('martanal_mobidity_round_visit.csv',index=False)